In [55]:
from archive_query_log import DATA_DIRECTORY_PATH

example_dir = DATA_DIRECTORY_PATH / "examples"
example_serps_path_json = example_dir / "serps.jsonl"
example_serps_dir = example_dir / "serps"
example_serps_dir_parquet = example_serps_dir / "parquet"
example_serps_dir_jsonl = example_serps_dir / "jsonl"
example_serps_dir_csv = example_serps_dir / "csv"
example_results_path_json = example_dir / "results.jsonl"
example_results_dir = example_dir / "results"
example_results_dir_parquet = example_results_dir / "parquet"
example_results_dir_jsonl = example_results_dir / "jsonl"
example_results_dir_csv = example_results_dir / "csv"

In [56]:
from archive_query_log.config import SERVICES
from datetime import datetime
from publicsuffixlist import PublicSuffixList
from urllib.parse import urlparse

public_suffix_list = PublicSuffixList()


def domain(url):
    return urlparse(url).hostname


def public_suffix(url):
    return public_suffix_list.publicsuffix(domain(url))


def year(timestamp):
    return datetime.fromtimestamp(timestamp).year


def month(timestamp):
    return datetime.fromtimestamp(timestamp).month


def search_provider(name):
    service = SERVICES[name]
    return {
        "name": name,
        "alexa_domain": service.alexa_domain,
        "alexa_public_suffix": service.public_suffix,
        "alexa_rank": service.alexa_rank,
        "category": service.category,
    }

In [57]:
from json import loads

with example_serps_path_json.open("rt") as example_serps_file:
    example_serps = [loads(line) for line in example_serps_file]

In [58]:
from pyarrow import table
from archive_query_log.schema import SERP_SCHEMA

serps_table = table(
    data={
        "serp_id": [query["id"] for query in example_serps],
        "serp_url": [query["url"] for query in example_serps],
        "serp_domain": [domain(query["url"]) for query in example_serps],
        "serp_domain_public_suffix": [public_suffix(query["url"]) for query in example_serps],
        "serp_timestamp": [query["timestamp"] for query in example_serps],
        "serp_year": [year(query["timestamp"]) for query in example_serps],
        "serp_month": [month(query["timestamp"]) for query in example_serps],
        "serp_wayback_url": [query["wayback_url"] for query in example_serps],
        "serp_wayback_raw_url": [query["wayback_raw_url"] for query in example_serps],
        "serp_page": [query["url_page"] for query in example_serps],
        "serp_offset": [query["url_offset"] for query in example_serps],
        "serp_query_text_url": [query["url_query"] for query in example_serps],
        "serp_query_text_url_language": [query["language"] for query in example_serps],
        "serp_query_text_html": [query["serp_query"] for query in example_serps],
        "serp_warc_relative_path": [query["archived_raw_serp_location"]["relative_path"] for query in example_serps],
        "serp_warc_byte_offset": [query["archived_raw_serp_location"]["byte_offset"] for query in example_serps],
        "serp_results": [
            [
                {
                    "result_id": result["id"],
                    "result_url": result["url"],
                    "result_domain": domain(result["url"]),
                    "result_domain_public_suffix": public_suffix(result["url"]),
                    "result_wayback_url": result["wayback_url"],
                    "result_wayback_raw_url": result["wayback_raw_url"],
                    "result_snippet_rank": result["snippet_rank"],
                    "result_snippet_title": result["snippet_title"],
                    "result_snippet_text": result["snippet_text"],
                    "result_warc_relative_path": None,
                    "result_warc_byte_offset": None,
                }
                for result in query["results"]
            ]
            for query in example_serps
        ],
        "search_provider_name": [query["service"] for query in example_serps],
        "search_provider_alexa_domain": [search_provider(query["service"])["alexa_domain"] for query in example_serps],
        "search_provider_alexa_domain_public_suffix":
            [search_provider(query["service"])["alexa_public_suffix"] for query in example_serps],
        "search_provider_alexa_rank": [search_provider(query["service"])["alexa_rank"] for query in example_serps],
        "search_provider_category": [search_provider(query["service"])["category"] for query in example_serps],
    },
    schema=SERP_SCHEMA,
)

In [59]:
from shutil import rmtree, make_archive
from archive_query_log.schema import SERP_PARTITIONING
from pyarrow.parquet import write_to_dataset
from pandas import read_parquet

if example_serps_dir.exists():
    rmtree(example_serps_dir)
example_serps_dir.mkdir()
write_to_dataset(
    serps_table,
    root_path=example_serps_dir_parquet,
    partitioning=SERP_PARTITIONING,
    basename_template="serps_{i}.parquet",
)
for path in example_serps_dir_parquet.rglob("**/*.parquet"):
    df = read_parquet(path)
    rel_path = path.relative_to(example_serps_dir_parquet)
    csv_path = (example_serps_dir_csv / rel_path).with_suffix(".csv")
    csv_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(csv_path, index=False)
    jsonl_path = (example_serps_dir_jsonl / rel_path).with_suffix(".json")
    jsonl_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_json(jsonl_path, orient="records", lines=True)
make_archive(example_serps_dir, "zip", example_serps_dir)

'/home/heinrich/Repositories/archive-query-log/data/examples/serps.zip'

In [60]:
read_parquet(example_serps_dir_parquet)

,serp_id,serp_url,serp_timestamp,serp_wayback_url,serp_wayback_raw_url,serp_page,serp_offset,serp_query_text_url,serp_query_text_url_language,serp_query_text_html,...,serp_results,search_provider_name,search_provider_alexa_domain,search_provider_alexa_domain_public_suffix,search_provider_alexa_rank,search_provider_category,serp_domain_public_suffix,serp_domain,serp_year,serp_month
0,3d8b4173-1cc6-5f4b-8ed3-30f10e439a42,https://www.google.co.jp/search?q=%22FK%E3%82%...,2022-01-03 02:14:44,https://web.archive.org/web/20220103031444/htt...,https://web.archive.org/web/20220103031444id_/...,NaN,NaN,"""FKジェレズニク"" -wikipedia",kk,"""FKジェレズニク"" -wikipedia",...,[{'result_id': 'ea1b3af7-ba83-59d3-87b0-c508ed...,google,google.com,com,1,search-engine,co.jp,www.google.co.jp,2022,1
1,1149f047-d43c-5e67-a34f-55614c2cc3ba,http://www.google.com.sg:80/search?hl=en&ie=IS...,2003-08-31 19:54:34,https://web.archive.org/web/20030831215434/htt...,https://web.archive.org/web/20030831215434id_/...,NaN,NaN,"""ASP"" ""Microsoft Access""",NaN,"""ASP"" ""Microsoft Access""",...,[{'result_id': 'c9661b3e-d736-5b81-a3fe-e97c4f...,google,google.com,com,1,search-engine,com.sg,www.google.com.sg,2003,8
2,b32f13d4-aa25-5ff5-b338-c9e3df466cb4,https://ja-jp.facebook.com/search.php?q=asano+...,2013-05-28 14:42:11,https://web.archive.org/web/20130528164211/htt...,https://web.archive.org/web/20130528164211id_/...,NaN,NaN,asano yuga,ja,asano yuga,...,[],facebook,facebook.com,com,5,social-media,com,ja-jp.facebook.com,2013,5
3,c027789e-30d9-5532-a6b9-bd1b73300ac9,https://mobile.twitter.com/search?q=Eurovision...,2014-10-19 06:53:18,https://web.archive.org/web/20141019085318/htt...,https://web.archive.org/web/20141019085318id_/...,NaN,NaN,EurovisionSongContest2014,ky,EurovisionSongContest2014,...,[{'result_id': 'e65e2155-ec92-5073-9d82-15cca0...,twitter,twitter.com,com,26,social-media,com,mobile.twitter.com,2014,10
4,8d6e45fb-bd0c-514d-890d-55802b52550a,https://s.weibo.com/weibo/%23songhyekyo%23,2021-04-20 15:47:32,https://web.archive.org/web/20210420174732/htt...,https://web.archive.org/web/20210420174732id_/...,NaN,NaN,#songhyekyo#,NaN,#songhyekyo#,...,[],weibo,weibo.com,com,15,social-media,com,s.weibo.com,2021,4
5,afbb1256-ffa0-5ba4-a450-6a89aa2f3f59,https://scholar.google.com/scholar?q=Tripanoso...,2019-06-20 02:45:46,https://web.archive.org/web/20190620044546/htt...,https://web.archive.org/web/20190620044546id_/...,NaN,NaN,Tripanosomas semelhantes do Trypanosoma cruzi ...,pt,Tripanosomas semelhantes do Trypanosoma cruzi ...,...,[],google,google.com,com,1,search-engine,com,scholar.google.com,2019,6
6,56cbbee2-9915-58d9-9141-a5690428f7e7,https://scholar.google.com/scholar?q=Genetic+e...,2020-02-23 09:02:01,https://web.archive.org/web/20200223100201/htt...,https://web.archive.org/web/20200223100201id_/...,NaN,NaN,Genetic exchange among bdelloid rotifers is mo...,NaN,Genetic exchange among bdelloid rotifers is mo...,...,[],google,google.com,com,1,search-engine,com,scholar.google.com,2020,2
7,c7d12165-1b24-5644-a95a-2af066e18996,https://scholar.google.com/scholar?q=author:%2...,2020-06-26 11:48:54,https://web.archive.org/web/20200626134854/htt...,https://web.archive.org/web/20200626134854id_/...,NaN,NaN,"author:""van der Wouden Johannes C""",nl,"author:""van der Wouden Johannes C""",...,[],google,google.com,com,1,search-engine,com,scholar.google.com,2020,6
8,80d05846-77cb-5892-8570-cf28567c975d,https://scholar.google.com/scholar?hl=en&q=P%C...,2021-02-25 22:30:47,https://web.archive.org/web/20210225233047/htt...,https://web.archive.org/web/20210225233047id_/...,NaN,NaN,Päivi Majaranta and Kari Jouko Räihä. 2002. Tw...,NaN,Päivi Majaranta and Kari Jouko Räihä. 2002. Tw...,...,[],google,google.com,com,1,search-engine,com,scholar.google.com,2021,2
9,d7899daf-8fee-51b1-a1e6-25a4c5995f67,https://scholar.google.com/scholar?hl=en&q=L%C...,2021-02-24 17:15:05,https://web.archive.org/web/20210224181505/htt...,https://web.archive.org/web/20210224181505id_/...,NaN,NaN,"Lützenberger, 

In [61]:
with example_results_path_json.open("rt") as example_results_file:
    example_results = [loads(line) for line in example_results_file]

In [62]:
from archive_query_log.schema import RESULT_SCHEMA

results_table = table(
    data={
        "result_id": [result["id"] for result in example_results],
        "result_url": [result["url"] for result in example_results],
        "result_domain": [domain(result["url"]) for result in example_results],
        "result_domain_public_suffix": [public_suffix(result["url"]) for result in example_results],
        "result_wayback_url": [result["wayback_url"] for result in example_results],
        "result_wayback_raw_url": [result["wayback_raw_url"] for result in example_results],
        "result_snippet_rank": [result["snippet_rank"] for result in example_results],
        "result_snippet_title": [result["snippet_title"] for result in example_results],
        "result_snippet_text": [result["snippet_text"] for result in example_results],
        "result_warc_relative_path": [None for result in example_results],
        "result_warc_byte_offset": [None for result in example_results],
        "serp_id": [result["query"]["id"] for result in example_results],
        "serp_url": [result["query"]["url"] for result in example_results],
        "serp_domain": [domain(result["query"]["url"]) for result in example_results],
        "serp_domain_public_suffix": [public_suffix(result["query"]["url"]) for result in example_results],
        "serp_timestamp": [result["query"]["timestamp"] for result in example_results],
        "serp_year": [year(result["query"]["timestamp"]) for result in example_results],
        "serp_month": [month(result["query"]["timestamp"]) for result in example_results],
        "serp_wayback_url": [result["query"]["wayback_url"] for result in example_results],
        "serp_wayback_raw_url": [result["query"]["wayback_raw_url"] for result in example_results],
        "serp_page": [result["query"]["url_page"] for result in example_results],
        "serp_offset": [result["query"]["url_offset"] for result in example_results],
        "serp_query_text_url": [result["query"]["url_query"] for result in example_results],
        "serp_query_text_url_language": [result["query"]["language"] for result in example_results],
        "serp_query_text_html": [result["query"]["serp_query"] for result in example_results],
        "serp_warc_relative_path":
            [result["query"]["archived_raw_serp_location"]["relative_path"] for result in example_results],
        "serp_warc_byte_offset":
            [result["query"]["archived_raw_serp_location"]["byte_offset"] for result in example_results],
        "search_provider_name": [result["query"]["service"] for result in example_results],
        "search_provider_alexa_domain":
            [search_provider(result["query"]["service"])["alexa_domain"] for result in example_results],
        "search_provider_alexa_domain_public_suffix":
            [search_provider(result["query"]["service"])["alexa_public_suffix"] for result in example_results],
        "search_provider_alexa_rank":
            [search_provider(result["query"]["service"])["alexa_rank"] for result in example_results],
        "search_provider_category":
            [search_provider(result["query"]["service"])["category"] for result in example_results],
    },
    schema=RESULT_SCHEMA,
)

In [63]:
from archive_query_log.schema import RESULT_PARTITIONING

if example_results_dir.exists():
    rmtree(example_results_dir)
example_results_dir.mkdir()
write_to_dataset(
    results_table,
    root_path=example_results_dir_parquet,
    partitioning=RESULT_PARTITIONING,
    basename_template="results_{i}.parquet",
)
for path in example_results_dir_parquet.rglob("**/*.parquet"):
    df = read_parquet(path)
    rel_path = path.relative_to(example_results_dir_parquet)
    csv_path = (example_results_dir_csv / rel_path).with_suffix(".csv")
    csv_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(csv_path, index=False)
    jsonl_path = (example_results_dir_jsonl / rel_path).with_suffix(".json")
    jsonl_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_json(jsonl_path, orient="records", lines=True)
make_archive(example_results_dir, "zip", example_results_dir)

'/home/heinrich/Repositories/archive-query-log/data/examples/results.zip'

In [64]:
from pandas import read_parquet

read_parquet(example_results_dir_parquet).sort_values(["serp_id", "result_snippet_rank"])

,result_id,result_url,result_domain,result_domain_public_suffix,result_wayback_url,result_wayback_raw_url,result_snippet_rank,result_snippet_title,result_snippet_text,result_warc_relative_path,...,serp_warc_byte_offset,search_provider_name,search_provider_alexa_domain,search_provider_alexa_domain_public_suffix,search_provider_alexa_rank,search_provider_category,serp_domain_public_suffix,serp_domain,serp_year,serp_month
30,3fc164de-4938-5b96-94d5-90b712c1f719,https://www.youtube.com/watch?v=GAgEkjDizyE,www.youtube.com,com,https://web.archive.org/web/20191025135059/htt...,https://web.archive.org/web/20191025135059id_/...,20,Search hidden 'R' found in the New World Loadi...,Support A Creator code - inthelittlewood Twitc...,None,...,57442749,youtube,youtube.com,com,2,media-sharing,com,www.youtube.com,2019,10
23,b97ced3d-709c-50d0-a503-64fcda7d33ac,https://rodaleinstitute.org/blog/what-bidens-c...,rodaleinstitute.org,org,https://web.archive.org/web/20210423062233/htt...,https://web.archive.org/web/20210423062233id_/...,10,What Biden's Climate Plan Means For Regenerati...,1 Feb 2021 — President Joe <em>Biden</em> has ...,None,...,420323,google,google.com,com,1,search-engine,com,www.google.com,2021,4
21,7faf2417-c890-5b48-b1c2-fbd5563807bb,https://www.google.com/url?q=https://hu.wikipe...,www.google.com,com,https://web.archive.org/web/20210114133654/htt...,https://web.archive.org/web/20210114133654id_/...,5,Angela Merkel – Wikipédia,None,None,...,2356073,google,google.com,com,1,search-engine,com,www.google.com,2021,1
44,ef8ee7c9-73d2-50cb-a1ed-0dac327eaf7f,https://twitter.com/nailartbd/status/138202747...,twitter.com,com,https://web.archive.org/web/20210909025649/htt...,https://web.archive.org/web/20210909025649id_/...,6,"Nail Art Express в Twitter: ""Just posted a vid...",<em>Nail Art Express</em> · @NailartBd. First ...,None,...,923796,google,google.com,com,1,search-engine,dk,www.google.dk,2021,9
13,888dc065-4733-5ec9-ad76-562dff5efa8c,https://stackoverflow.com/questions/43366627/c...,stackoverflow.com,com,https://web.archive.org/web/20200518111006/htt...,https://web.archive.org/web/20200518111006id_/...,6,Cracks in webaudio playback during streaming o...,I have a server sending chunks of raw audio ov...,None,...,451128,stackoverflow,stackoverflow.com,com,43,question-and-answer,com,stackoverflow.com,2020,5
3,c0cd23e0-ae49-5b6e-a551-dbbf8fc8b9c2,https://www.ebay.com.au/itm/DIY-LED-Dash-Clust...,www.ebay.com.au,com.au,https://web.archive.org/web/20200108102227/htt...,https://web.archive.org/web/20200108102227id_/...,28,DIY LED Dash Cluster Light Bulbs VT VX VU WHCo...,None,None,...,60569302,ebay,ebay.com,com,31,e-commerce,com.au,www.ebay.com.au,2020,1
38,e556ac4e-7137-5c35-8367-bb0d611d2e2f,https://www.youtube.com/watch?v=vjYxQ-v_q-8,www.youtube.com,com,https://web.archive.org/web/20200114111916/htt...,https://web.archive.org/web/20200114111916id_/...,21,Основные ошибки бариста в работе с кофемашиной...,Наш новый ролик посвящен основным ошибкам бари...,None,...,152074,youtube,youtube.com,com,2,media-sharing,com,www.youtube.com,2020,1
46,d93721f5-4737-51c3-8d47-0671bc876a29,https://www.ebay.fr/itm/332449231778?hash=item...,www.ebay.fr,fr,https://web.archive.org/web/20211026163403/htt...,https://web.archive.org/web/20211026163403id_/...,61,CRYSTAL AURUM 2X 001 AUR2 Swarovski 5328 Xilio...,None,None,...,162515,ebay,ebay.com,com,31,e-commerce,fr,www.ebay.fr,2021,10
41,b0ec792a-7b58-574b-af5c-6a84cae5270c,https://www.youtube.com/watch?v=ykb_4-Dckdc,www.youtube.com,com,https://web.archive.org/web/20200131213730/htt...,https://web.archive.org/web/20200131213730id_/...,10,Batman Arkham | 1 сезон | 1 серия - Продолжите...,None,None,...,915447,youtube,youtube.com,com,2,media-sharing,com,www.youtube.com,2020,1
6,0213be68-9ff2-5e8d-a5dc-20af24d49cb5,https://mobile.twitter.com/iPhoneDev_Me/status...,mobile.twitter.com,com,https://web.archive.org/web/20140920090721/htt...,https://web.archive.org/web/20140920090721id_/...,15,<em>iPhoneDeveloper.Me<